# Module 10: Multi-Timeframe Analysis\n\n---\n\n## Welcome to Module 10!\n\nProfessional traders don't look at just one timeframe. They analyze daily, weekly, and monthly charts to get the complete picture!\n\n### Learning Objectives\n\n- Understand the importance of multiple timeframes\n- Apply top-down analysis approach\n- Identify trends across different timeframes\n- Use higher timeframes for direction, lower for entry\n- Avoid conflicting signals\n\n### Prerequisites\n\n- Completed Modules 00-09\n- Understanding of trend analysis\n- Familiarity with support/resistance\n\n### Time Required: 60-75 minutes\n\n---

## Part 1: Why Multiple Timeframes?\n\n### The Single Timeframe Problem\n\nLooking at only one timeframe is like:\n- Reading a book through a keyhole (missing the big picture)\n- Zooming too far into a map (can't see where you're going)\n\n### The Multi-Timeframe Solution\n\n**Top-Down Analysis**:\n1. **Monthly** = Long-term direction (forest view)\n2. **Weekly** = Medium-term trend (trees view)\n3. **Daily** = Short-term entry/exit (leaves view)\n\n### The Golden Rule\n\n**"Trade in the direction of the higher timeframe trend"**\n\n- Monthly uptrend → Look for buy setups on daily\n- Weekly downtrend → Look for sell setups on daily\n- Don't fight the higher timeframe!\n\n### Real-World Analogy\n\nThink of timeframes like **planning a road trip**:\n- **Monthly chart** = Country map (overall direction: north or south?)\n- **Weekly chart** = State map (which highways to take?)\n- **Daily chart** = City map (exact streets and turns)\n\nYou need ALL THREE to reach your destination successfully!\n\n---

In [ ]:
# Import libraries\nimport yfinance as yf\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport plotly.graph_objects as go\nfrom plotly.subplots import make_subplots\nimport warnings\n\nwarnings.filterwarnings('ignore')\n%matplotlib inline\n\ntry:\n    plt.style.use('seaborn-v0_8-darkgrid')\nexcept:\n    try:\n        plt.style.use('seaborn-darkgrid')\n    except:\n        plt.style.use('default')\n\nprint("Libraries imported successfully!")

In [ ]:
# Fetch data for different timeframes\nticker = "0310.KL"\nprint(f"Fetching multi-timeframe data for {ticker}...\\n")\n\n# Daily data (1 year)\ndaily_data = yf.download(ticker, period="1y", interval="1d", progress=False)\nif isinstance(daily_data.columns, pd.MultiIndex):\n    daily_data.columns = daily_data.columns.get_level_values(0)\n\n# Weekly data (2 years)\nweekly_data = yf.download(ticker, period="2y", interval="1wk", progress=False)\nif isinstance(weekly_data.columns, pd.MultiIndex):\n    weekly_data.columns = weekly_data.columns.get_level_values(0)\n\n# Monthly data (5 years)\nmonthly_data = yf.download(ticker, period="5y", interval="1mo", progress=False)\nif isinstance(monthly_data.columns, pd.MultiIndex):\n    monthly_data.columns = monthly_data.columns.get_level_values(0)\n\nprint(f"Daily: {len(daily_data)} bars")\nprint(f"Weekly: {len(weekly_data)} bars")\nprint(f"Monthly: {len(monthly_data)} bars")\nprint(f"\\nCurrent price: RM {daily_data['Close'].iloc[-1]:.2f}")

## Part 2: Analyzing Trends Across Timeframes\n\n### Trend Identification\n\nWe'll use moving averages to identify trends:\n- **50 SMA** = Short-term trend\n- **200 SMA** = Long-term trend\n- **Price above both** = Strong uptrend\n- **Price below both** = Strong downtrend\n\n---

In [ ]:
# Calculate moving averages for each timeframe\ndef add_trend_indicators(df, name):\n    """Add SMA 50 and 200 to dataframe"""\n    df['SMA_50'] = df['Close'].rolling(window=50).mean()\n    df['SMA_200'] = df['Close'].rolling(window=200).mean()\n    \n    # Determine trend\n    latest_close = df['Close'].iloc[-1]\n    latest_sma50 = df['SMA_50'].iloc[-1]\n    latest_sma200 = df['SMA_200'].iloc[-1]\n    \n    if pd.notna(latest_sma50) and pd.notna(latest_sma200):\n        if latest_close > latest_sma50 > latest_sma200:\n            trend = "STRONG UPTREND"\n        elif latest_close > latest_sma50 or latest_close > latest_sma200:\n            trend = "UPTREND"\n        elif latest_close < latest_sma50 < latest_sma200:\n            trend = "STRONG DOWNTREND"\n        elif latest_close < latest_sma50 or latest_close < latest_sma200:\n            trend = "DOWNTREND"\n        else:\n            trend = "NEUTRAL"\n    else:\n        trend = "INSUFFICIENT DATA"\n    \n    print(f"{name} Timeframe:")\n    print(f"  Price: RM {latest_close:.2f}")\n    if pd.notna(latest_sma50):\n        print(f"  SMA 50: RM {latest_sma50:.2f}")\n    if pd.notna(latest_sma200):\n        print(f"  SMA 200: RM {latest_sma200:.2f}")\n    print(f"  Trend: {trend}\\n")\n    \n    return trend\n\n# Analyze all timeframes\nprint("Multi-Timeframe Trend Analysis\\n")\nprint("="*60)\n\nmonthly_trend = add_trend_indicators(monthly_data, "MONTHLY")\nweekly_trend = add_trend_indicators(weekly_data, "WEEKLY")\ndaily_trend = add_trend_indicators(daily_data, "DAILY")\n\nprint("="*60)\nprint("TIMEFRAME ALIGNMENT:")\n\n# Check alignment\nif "UPTREND" in monthly_trend and "UPTREND" in weekly_trend and "UPTREND" in daily_trend:\n    print("ALL BULLISH - Perfect alignment for long positions!")\nelif "DOWNTREND" in monthly_trend and "DOWNTREND" in weekly_trend and "DOWNTREND" in daily_trend:\n    print("ALL BEARISH - Perfect alignment for short positions!")\nelif "UPTREND" in monthly_trend and "UPTREND" in weekly_trend:\n    print("Higher timeframes BULLISH - Look for buy opportunities on daily pullbacks")\nelif "DOWNTREND" in monthly_trend and "DOWNTREND" in weekly_trend:\n    print("Higher timeframes BEARISH - Look for sell opportunities on daily rallies")\nelse:\n    print("MIXED SIGNALS - Timeframes not aligned, trade with caution!")

In [ ]:
# Visualize multiple timeframes\nfig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=False)\n\n# Monthly chart\naxes[0].plot(monthly_data.index, monthly_data['Close'], label='Close', linewidth=2, color='black')\naxes[0].plot(monthly_data.index, monthly_data['SMA_50'], label='SMA 50', linestyle='--', color='blue')\naxes[0].plot(monthly_data.index, monthly_data['SMA_200'], label='SMA 200', linestyle='--', color='red')\naxes[0].set_title(f'{ticker} - MONTHLY Chart', fontsize=14, fontweight='bold')\naxes[0].set_ylabel('Price (RM)')\naxes[0].legend(loc='best')\naxes[0].grid(True, alpha=0.3)\naxes[0].text(0.02, 0.95, f'Trend: {monthly_trend}', transform=axes[0].transAxes, \n            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5), verticalalignment='top')\n\n# Weekly chart\naxes[1].plot(weekly_data.index, weekly_data['Close'], label='Close', linewidth=2, color='black')\naxes[1].plot(weekly_data.index, weekly_data['SMA_50'], label='SMA 50', linestyle='--', color='blue')\naxes[1].plot(weekly_data.index, weekly_data['SMA_200'], label='SMA 200', linestyle='--', color='red')\naxes[1].set_title(f'{ticker} - WEEKLY Chart', fontsize=14, fontweight='bold')\naxes[1].set_ylabel('Price (RM)')\naxes[1].legend(loc='best')\naxes[1].grid(True, alpha=0.3)\naxes[1].text(0.02, 0.95, f'Trend: {weekly_trend}', transform=axes[1].transAxes,\n            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5), verticalalignment='top')\n\n# Daily chart\naxes[2].plot(daily_data.index, daily_data['Close'], label='Close', linewidth=2, color='black')\naxes[2].plot(daily_data.index, daily_data['SMA_50'], label='SMA 50', linestyle='--', color='blue')\naxes[2].plot(daily_data.index, daily_data['SMA_200'], label='SMA 200', linestyle='--', color='red')\naxes[2].set_title(f'{ticker} - DAILY Chart', fontsize=14, fontweight='bold')\naxes[2].set_ylabel('Price (RM)')\naxes[2].set_xlabel('Date')\naxes[2].legend(loc='best')\naxes[2].grid(True, alpha=0.3)\naxes[2].text(0.02, 0.95, f'Trend: {daily_trend}', transform=axes[2].transAxes,\n            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5), verticalalignment='top')\n\nplt.tight_layout()\nplt.show()\n\nprint("\\nCompare the three charts to see how trends differ across timeframes")

---\n\n## Part 3: Multi-Timeframe Trading Strategies\n\n### Strategy 1: Top-Down Analysis\n\n**Steps**:\n1. Check monthly trend (long-term direction)\n2. Check weekly trend (medium-term confirmation)\n3. Look for entry on daily chart\n\n**Rules**:\n- Monthly + Weekly uptrend → Only look for BUY setups on daily\n- Monthly + Weekly downtrend → Only look for SELL setups on daily\n- Mixed signals → Stay out or reduce position size\n\n### Strategy 2: Pullback Entry\n\n**Example (Bullish)**:\n1. Monthly = Uptrend\n2. Weekly = Uptrend\n3. Daily = Pullback to support (50 SMA)\n4. ENTRY: When daily bounces off support\n5. STOP: Below recent swing low\n6. TARGET: Weekly resistance or higher\n\n### Strategy 3: Trend Confirmation\n\n**Best Scenario**:\n- Monthly: UPTREND\n- Weekly: UPTREND\n- Daily: UPTREND\n- **Action**: Aggressive long positions\n\n**Worst Scenario**:\n- Monthly: UPTREND\n- Weekly: DOWNTREND\n- Daily: UPTREND\n- **Action**: Stay out (conflicting signals)\n\n---

## Key Takeaways\n\n### What You Learned\n\n1. **Multiple Timeframes Matter**\n   - Never trade on single timeframe\n   - Higher timeframe = overall direction\n   - Lower timeframe = entry/exit timing\n\n2. **Top-Down Analysis**\n   - Start with monthly (big picture)\n   - Confirm with weekly (medium trend)\n   - Execute on daily (precise entry)\n\n3. **Trend Alignment**\n   - All timeframes aligned = Strongest signals\n   - Mixed signals = Stay out or reduce size\n   - Trade WITH higher timeframe trend\n\n4. **Best Practices**\n   - Monthly for direction\n   - Weekly for confirmation\n   - Daily for entry\n   - Don't fight the higher timeframe!\n\n---\n\n## Practice Exercise\n\nTry analyzing different Malaysian stocks:\n- 1155.KL (Maybank)\n- 5296.KL (Tenaga)\n\nFor each:\n1. Identify monthly trend\n2. Identify weekly trend\n3. Identify daily trend\n4. Are they aligned?\n5. What position would you take (long/short/none)?\n\n---\n\n## What's Next?\n\n**Module 11: Advanced Chart Patterns & Trend Strength**\n\nLearn to identify powerful chart patterns and measure trend strength!\n\n---